In [1]:
import pandas as pd

from ddf_utils.str import to_concept_id

In [2]:
cb_csv = '../source/IHME_GLOBAL_EDUCATIONAL_ATTAINMENT_1970_2015/IHME_GLOBAL_EDUCATIONAL_ATTAINMENT_1970_2015_CB_Y2015M04D27.CSV'
data_csv = '../source/IHME_GLOBAL_EDUCATIONAL_ATTAINMENT_1970_2015/IHME_GLOBAL_EDUCATIONAL_ATTAINMENT_1970_2015_Y2015M04D27.CSV'

In [3]:
cb = pd.read_csv(cb_csv, skiprows=1)

In [4]:
cb = cb.drop('Variable:', axis=1)

In [5]:
cb.head()

,location_id,location_code,location_name,year,age_group_id,age_group_name,sex_id,sex_name,metric,unit,mean,upper,lower
0,Location ID,"Country ISO 3166-1 Alpha-3 Code, GBD Region Co...","Country, Region, or Super-Region Name",Year,Age Group ID,Age Group Name,Sex ID,Sex Name,Metric,Unit,Mean,95% Uncertainty Interval - Upper Bound,95% Uncertainty Interval - Lower Bound
1,1,AFG,Afghanistan,1970,149,15 to 24,1,Males,Age Standardized Education Per Capita,Years,NaN,NaN,NaN
2,4,AGO,Albania,1971,150,15 to 44,2,Females,Education Per Capita,NaN,NaN,NaN,NaN
3,5,ALB,Algeria,1972,151,25 plus,3,Both,Population Weighted Education Per Capita,NaN,NaN,NaN,NaN
4,6,AND,Andean Latin America,1973,152,25 to 34,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data = pd.read_csv(data_csv)

In [7]:
data.head()

,location_id,location_code,location_name,year,age_group_id,age_group_name,sex_id,sex_name,metric,unit,mean,upper,lower
0,160,AFG,Afghanistan,1970,156,15 to 44,2,Females,Age Standardized Education Per Capita,Years,0.29,0.55,0.15
1,160,AFG,Afghanistan,1970,157,25 plus,1,Males,Age Standardized Education Per Capita,Years,1.03,1.66,0.63
2,160,AFG,Afghanistan,1970,157,25 plus,2,Females,Age Standardized Education Per Capita,Years,0.13,0.24,0.07
3,160,AFG,Afghanistan,1970,157,25 plus,3,Both,Age Standardized Education Per Capita,Years,0.58,0.95,0.35
4,168,AGO,Angola,1970,156,15 to 44,2,Females,Age Standardized Education Per Capita,Years,1.41,2.29,0.85


In [8]:
data[(data.location_code == 'AFG') & (data.age_group_id == 153) & (data.year == 1983)][
    ['location_code', 'year', 'age_group_name', 'sex_name', 'metric', 'mean', 'upper', 'lower']]

,location_code,year,age_group_name,sex_name,metric,mean,upper,lower
73788,AFG,1983,55 to 64,Males,Education Per Capita,1.07,1.71,0.67
73789,AFG,1983,55 to 64,Females,Education Per Capita,0.09,0.16,0.05


In [9]:
data.shape

(199640, 13)

In [11]:
# datapoints

In [12]:
data['metric'] = data['metric'].map(to_concept_id)

In [13]:
data.head()

,location_id,location_code,location_name,year,age_group_id,age_group_name,sex_id,sex_name,metric,unit,mean,upper,lower
0,160,AFG,Afghanistan,1970,156,15 to 44,2,Females,age_standardized_education_per_capita,Years,0.29,0.55,0.15
1,160,AFG,Afghanistan,1970,157,25 plus,1,Males,age_standardized_education_per_capita,Years,1.03,1.66,0.63
2,160,AFG,Afghanistan,1970,157,25 plus,2,Females,age_standardized_education_per_capita,Years,0.13,0.24,0.07
3,160,AFG,Afghanistan,1970,157,25 plus,3,Both,age_standardized_education_per_capita,Years,0.58,0.95,0.35
4,168,AGO,Angola,1970,156,15 to 44,2,Females,age_standardized_education_per_capita,Years,1.41,2.29,0.85


In [14]:
data.metric.unique()

array(['age_standardized_education_per_capita', 'education_per_capita',
       'population_weighted_education_per_capita'], dtype=object)

In [15]:
met = 'population_weighted_education_per_capita'

d1 = data.groupby(by='metric').get_group(met)

In [16]:
for g, idxs in d1.groupby('age_group_name').groups.items():
    df = d1.ix[idxs].copy()
    for i in ['mean', 'lower', 'upper']:
        name = '{}_{}_{}'.format(met, to_concept_id(g), i)
        df = df.rename(columns={i: name})
        df[['location_id', 'year', 'sex_id', name]].to_csv(
            '../../ddf--datapoints--{}--by--location_id-sex_id--year.csv'.format(name), 
            index=False, float_format='%.2f')

In [17]:
data.metric.unique()

array(['age_standardized_education_per_capita', 'education_per_capita',
       'population_weighted_education_per_capita'], dtype=object)

In [18]:
# These 2 lists are for later uses in concept tables

names = list()
measures = list()

for met in cb.metric.drop(0).dropna().unique():
    met_id = to_concept_id(met)
    d1 = data.groupby(by='metric').get_group(met_id)
    for g, idxs in d1.groupby('age_group_name').groups.items():
#         df = d1.ix[idxs].copy()
        for i in ['mean', 'lower', 'upper']:
            measure = '{}_{}_{}'.format(met_id, to_concept_id(g), i)
            measures.append(measure)
            
            if i in ['lower', 'upper']:
                name = '{}, Age {}: 95% Uncertainty Interval - {} Bound'.format(met, g, i.title())
            else:
                name = '{}, Age {}: Mean'.format(met, g)
            
            names.append(name)

In [19]:
names

['Age Standardized Education Per Capita, Age 25 plus: Mean',
 'Age Standardized Education Per Capita, Age 25 plus: 95% Uncertainty Interval - Lower Bound',
 'Age Standardized Education Per Capita, Age 25 plus: 95% Uncertainty Interval - Upper Bound',
 'Age Standardized Education Per Capita, Age 15 to 44: Mean',
 'Age Standardized Education Per Capita, Age 15 to 44: 95% Uncertainty Interval - Lower Bound',
 'Age Standardized Education Per Capita, Age 15 to 44: 95% Uncertainty Interval - Upper Bound',
 'Education Per Capita, Age 35 to 44: Mean',
 'Education Per Capita, Age 35 to 44: 95% Uncertainty Interval - Lower Bound',
 'Education Per Capita, Age 35 to 44: 95% Uncertainty Interval - Upper Bound',
 'Education Per Capita, Age 25 to 34: Mean',
 'Education Per Capita, Age 25 to 34: 95% Uncertainty Interval - Lower Bound',
 'Education Per Capita, Age 25 to 34: 95% Uncertainty Interval - Upper Bound',
 'Education Per Capita, Age 15 to 24: Mean',
 'Education Per Capita, Age 15 to 24: 95% Un

In [20]:
# entities

In [21]:
# location

loc = data[['location_id', 'location_code', 'location_name']].drop_duplicates()

In [22]:
loc.to_csv('../../ddf--entities--location_id.csv', index=False)

In [23]:
# sex

sex = data[['sex_id', 'sex_name']].drop_duplicates()

In [24]:
sex.to_csv('../../ddf--entities--sex_id.csv', index=False)

In [25]:
# concepts

In [26]:
allcol = cb.ix[0].T

In [27]:
allcol

location_id                                             Location ID
location_code     Country ISO 3166-1 Alpha-3 Code, GBD Region Co...
location_name                 Country, Region, or Super-Region Name
year                                                           Year
age_group_id                                           Age Group ID
age_group_name                                       Age Group Name
sex_id                                                       Sex ID
sex_name                                                   Sex Name
metric                                                       Metric
unit                                                           Unit
mean                                                           Mean
upper                        95% Uncertainty Interval - Upper Bound
lower                        95% Uncertainty Interval - Lower Bound
Name: 0, dtype: object

In [28]:
noneed = ['age_group_id', 'age_group_name', 'metric', 'mean', 'unit', 'upper', 'lower']

In [29]:
concepts = allcol[~allcol.index.isin(noneed)].reset_index()

In [30]:
concepts.columns = ['concept', 'name']
concepts

,concept,name
0,location_id,Location ID
1,location_code,"Country ISO 3166-1 Alpha-3 Code, GBD Region Co..."
2,location_name,"Country, Region, or Super-Region Name"
3,year,Year
4,sex_id,Sex ID
5,sex_name,Sex Name


In [31]:
concepts = concepts.append(pd.DataFrame({'concept': measures, 'name': names}))

In [32]:
concepts = concepts.set_index('concept')

In [33]:
concepts.ix[measures, 'concept_type'] = 'measure'
concepts.ix[['location_id', 'sex_id'], 'concept_type'] = 'entity_domain'
concepts.ix[['location_code', 'location_name', 'sex_name'], 'concept_type'] = 'string'
concepts.ix['year', 'concept_type'] = 'time'

In [34]:
concepts.ix['name', ['concept_type', 'name']] = ['string', 'Name']

In [35]:
concepts.to_csv('../../ddf--concepts.csv')

In [37]:
!validate-ddf ../../

[
{}]

